## Pulizia dei dati forniti dal docente per l'inserimento nel database

In [18]:
import pandas as pd
from pathlib import Path
from scripts.normalization import normalize_description, normalize_score, normalize_title

In [19]:
base_dir = Path().resolve().parent.parent
data_path = base_dir / 'data'
data_clean_path = base_dir / 'data_clean'
data={}

In [20]:
for file in data_path.glob('*.csv'):
    nome = file.stem
    data[nome] = pd.read_csv(file)

Per prima cosa, eliminiamo tutti i film che non hanno nome valido:

In [21]:
data['the_oscar_awards'] = data['the_oscar_awards'].dropna(subset='film')
data['rotten_tomatoes_reviews'] = data['rotten_tomatoes_reviews'].dropna(subset='movie_title')

Normalizzazione descrizioni e recensioni di rotten_tomato:

In [22]:
data['rotten_tomatoes_reviews'].review_score = data['rotten_tomatoes_reviews'].review_score.apply(normalize_score)
data['movies']['description'] = normalize_description(data['movies']['description'])
data['movies'].date =  data['movies'].date.astype('Int64')

Matching ID tra rotten_tomatoes_reviews, the_oscar_awards e movies.id:

In [23]:
data['movies']['name_clean'] = data['movies']['name'].apply(normalize_title)
data['rotten_tomatoes_reviews']['movie_name_clean'] = data['rotten_tomatoes_reviews']['movie_title'].apply(lambda x: normalize_title(x, parentesi=True))
data['the_oscar_awards']['film_clean'] = data['the_oscar_awards']['film'].apply(lambda x: normalize_title(x, parentesi=True))

In [24]:
movies_unique = data['movies'].drop_duplicates(subset='name_clean')

In [25]:
data['rotten_tomatoes_reviews'] = data['rotten_tomatoes_reviews'].merge(movies_unique[['id', 'name_clean']], left_on='movie_name_clean', right_on='name_clean', how='left').drop(columns=['movie_name_clean','name_clean'])
data['the_oscar_awards'] = data['the_oscar_awards'].merge(movies_unique[['id', 'name_clean']], left_on='film_clean', right_on='name_clean', how='left').drop(columns=['film_clean','name_clean'])
data['movies'] = data['movies'].drop(columns='name_clean')

In [26]:
data['the_oscar_awards'].id = data['the_oscar_awards'].id.astype('Int64')
data['rotten_tomatoes_reviews'].id = data['rotten_tomatoes_reviews'].id.astype('Int64')

In [27]:
data_clean_path.mkdir(exist_ok=True)

for name,df in data.items():
    if name != 'movies':
        df= df.rename(columns={'id':'id_movie'})

    df.to_csv(data_clean_path / f'{name}.csv', index=False)